# 데이터탐색

In [ ]:
#저장방법
df.to_csv('voc_rcp_practice2.csv', index=False)
# 첫번째 컬럼 "voc_trt_perd_itg_cd" 데이터 확인 및 값 분포 확인
df['voc_trt_perd_itg_cd'].value_counts()
# 첫번째 컬럼 "voc_trt_perd_itg_cd" 값 분포 비율 확인
df['voc_trt_perd_itg_cd'].value_counts(normalize=True)
# '_' 데이터 비율이 54% 차지하는 것으로 확인됨으로 해당 컬럼은 학습 데이터로 적합하지 않음으로 삭제
df = df.drop(columns=['voc_trt_perd_itg_cd'])
df.drop(columns=['new_date', 'opn_nfl_chg_date', 'cont_fns_pam_date'], inplace=True)
#"_" 항목을 NaN 값으로 치환
df['age_itg_cd'] = df['age_itg_cd'].replace("_", np.NaN)
df=df.astype({'age_itg_cd': int})
df['age_itg_cd'].isnull().sum()
# 'age_itg_cd' 항목의 type 변경
# NaN의 경우 int type을 지원하지 않아 float type으로 변경
df=df.astype({'age_itg_cd': float})
#info 로 확인
# 40% 이상 이상한 데이터 있는거 찾기
[df[c].value_counts(normalize=True) for c in df]
# 모든 _ 싹 바꿔버리는 코드
df_temp = df.replace('_', np.nan)
df_temp.isnull().sum()
# 싹 지워버리는 코드
df.drop(columns=['voc_trt_reslt_itg_cd',
                'oos_cause_type_itg_cd',
                'engt_cperd_type_itg_cd',
                'engt_tgt_div_itg_cd',
                'fclt_oos_yn',
                ], inplace=True)
# info 보고 형식 바꾸는 방법
df = df.astype({'voc_prod_sbt_id': object,
               'voc_wjt_sorc_id':object,
               'voc_type_itg_cd':object,
               'voc_sttus_itg_cd':object,
               'bprod_sbt_id':object,
               'voc_trt_degr_div_itg_cd':object,
               'voc_trt_need_time_itg_cd':object})

# 데이터전처리

In [ ]:
# KNNImputer ( 대신 가변수화 먼저 진행하고 해야함)
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(x)
# 데이터프레임으로 다시 만듭시다.
x = pd.DataFrame(imputed, columns=x.columns)
x.isna().sum()

In [ ]:
# df Data 복사
df_fix = df.copy()
#fillna 함수를 사용해서 특정 숫자나 문자로 결측치를 처리하는 방법
df=df.fillna(15)
df=df.fillna(method='backfill') 
df=df.fillna(method='ffill')
df['age_itg_cd']=df['age_itg_cd'].replace(np.nan, df['age_itg_cd'].median())
df=df.dropna() #listwise 방식으로 제거하기 record의 항목 중 1개의 값이라도 NA이면 해당 데이터 행 전체 제거
df=df.dropna(how='all') #pairwise 방식으로 제거하기 모든 항목이 NA인 데이터 행만 제거
df=df.dropna(thresh=14) # 14개이상이면 제거
df=df.dropna(subset=['cust_clas_itg_cd'])  # 특정열에 있는 na 만제거

In [ ]:
#이상치제거
df_data=df[(df['cust_clas_itg_cd']!='R')]
# 이상치변경
df_data['cust_clas_itg_cd']=df_data['cust_clas_itg_cd'].replace('R','L')
# 보통 value_counts()로 최빈값을 뽑아서 R을 L로 바꾸는 형식


In [ ]:
# 시각화
# 박스플롯
plt.figure(figsize=(16,6))
sns.boxplot(df['age_itg_cd'])
plt.show()
# 이상치 제거
#Q1, Q3 구하기
q1 = df['age_itg_cd'].quantile(0.25)
q3 = df['age_itg_cd'].quantile(0.75)
# 1.5 * IQR(Q3 - Q1)
iqr = 1.5 * (q3 - q1) 
# 이상치 제거하기
df_data=df[(df['age_itg_cd'] < (q3 + iqr)) & (df['age_itg_cd'] > (q1 - iqr))]
# 이상치 변경하기(최대 최소)
age_min = q1 - iqr
age_max = q3 + iqr
# 이상치를 변경
df[(df['age_itg_cd'] > age_max)]= age_max
df[(df['age_itg_cd'] < age_min)]= age_min

In [ ]:
# 최빈값 찾기
print(df['cont_sttus_itg_cd'].value_counts())
print(df['cust_dtl_ctg_itg_cd'].value_counts())
print(df['voc_mis_pbls_yn'].value_counts())

In [ ]:
#시각화
# Seaborn을 사용하여 간단한 차트를 그리기
plt.figure()
sns.lineplot([1,2,3], [100,120,110])
plt.show()
# scatter
plt.figure(figsize=(6,6))
plt.scatter(y='cust_snsry_base_conf_need_time', x='voc_trt_need_time_itg_cd', data=df)
plt.show()
# [Hint] 빈도는 파라미터 bins 활용 20개
plt.figure()
sns.histplot(x='age_itg_cd', bins=20 , data= df)
plt.xlabel('나이')
plt.ylabel('고객 수')
plt.show()
#나이대별 총이용금액 분포를 박스 그래프로 그리기
plt.boxplot(df['age_itg_cd'])
plt.show()
#컬럼별 상과관계를 heatmap 그래프로 그리기
plt.figure(figsize=(12, 12))
sns.heatmap(df.corr(), annot=True)
plt.show()
#두개선을 가지는 임의의 선그래프를 그리고 범례 추가하기
plt.plot([1,2,3], [1,4,9])
plt.plot([2,3,4],[5,6,7])
plt.xlabel('Quarter')
plt.ylabel('Score')
plt.title('Game Result')
plt.legend(['A team', 'B team'])
plt.show()

In [ ]:
# 'age_itg_cd'를 활describe 나이대("by_age")  Feature 만들기  (그냥 50대, 40대 칸 추가)
df['by_age']=df['age_itg_cd']//10*10
df=df.astype({'age_itg_cd': int, 'by_age':int})

In [ ]:
# 라이브러리 임포트
from sklearn.preprocessing import LabelEncoder
# LabelEncoder() 호출
le = LabelEncoder()
df['le_cust_clas_itg_cd'] = le.fit_transform(df['cust_clas_itg_cd'])
le.classes_

In [ ]:
object_columns = ['voc_type_itg_cd',
                  'voc_sttus_itg_cd',
                 'cust_clas_itg_cd',
                 'cont_sttus_itg_cd',
                 'cust_dtl_ctg_itg_cd',
                 'voc_trt_degr_div_itg_cd',
                 'voc_trt_need_time_itg_cd']
df = pd.get_dummies(df, columns=object_columns, drop_first=True)

# 머신러닝

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [ ]:
# 데이터 분리
X = df.drop(columns=['trm_yn'])
y = df['trm_yn']
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.3, stratify=y, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
x_test_std = scaler.transform(X_test)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# 모델 별 accuracy, precission , recall, f1_score를 List 로 저장
perfomance_acc = [accuracy_score(y_test, lr_pred), accuracy_score(y_test, knn_pred), accuracy_score(y_test, dt_pred), accuracy_score(y_test, rfc_pred)]
perfomance_precision = [precision_score(y_test, lr_pred), precision_score(y_test, knn_pred), precision_score(y_test, dt_pred), precision_score(y_test, rfc_pred)]
perfomance_recall = [recall_score(y_test, lr_pred), recall_score(y_test, knn_pred), recall_score(y_test, dt_pred), recall_score(y_test, rfc_pred)]
perfomance_f1_score = [f1_score(y_test, lr_pred), f1_score(y_test, knn_pred), f1_score(y_test, dt_pred), f1_score(y_test, rfc_pred)]
performance_index = ['accuary', 'precission', 'recall', 'f1_score']
performance_columns = ['Logistic', 'KNN', 'Decision', 'RandomForest']
# 모델 별 전체 성능을 데이터프레임으로 저장ㅌ
performance_total= pd.DataFrame(data=[perfomance_acc, perfomance_precision, perfomance_recall, perfomance_f1_score], index=performance_index, columns=performance_columns)

In [ ]:
# 모델당 정확도
plt.figure(figsize=(8,4))
plt.title('Accuracy')
sns.barplot(x=perfomance_acc, y=performance_columns)
plt.show()

In [ ]:
# rfc 중요도
feature = df.drop(columns=['trm_yn'])
rfc_importances_values = rfc_model.feature_importances_
rfc_importances = pd.Series(rfc_importances_values, index=feature.columns )
rfc_top10 = rfc_importances.sort_values(ascending=False)[:10]
plt.figure(figsize=(8,6))
plt.title('Top 10 Feature Importances')
sns.barplot(x=rfc_top10, y = rfc_top10.index)
plt.show()

In [ ]:
# KNN
model = KNeighborsClassifier(n_neighbors=2)
cv_score = cross_val_score(model, x_train_s, y_train, cv= 10)
result = {}
result['KNN'] = cv_score.mean()
#DecisionTree
model = DecisionTreeClassifier(random_state=2022)
cv_score = cross_val_score(model, x_train, y_train, cv=10)
result['Decision Tree'] = cv_score.mean()
#LogisticRegression
model = LogisticRegression(max_iter=500)
cv_score = cross_val_score(model, x_train_s, y_train, cv=10)
result['Logistic Regression'] = cv_score.mean()
# SVM
model = SVC(random_state=2022)
cv_score = cross_val_score(model, x_train_s, y_train, cv=10)
result['SVM'] = cv_score.mean()
# RandomForest
model = RandomForestClassifier(random_state=2022)
cv_score = cross_val_score(model, x_train, y_train, cv=10)
result['Random Forest'] = cv_score.mean()
#XGB
model = XGBClassifier(eval_metric='mlogloss',random_state=2022)
cv_score = cross_val_score(model, x_train, y_train, cv=10)
result['XGBoost'] = cv_score.mean()
# 평가 결과 확인
for m_name, m_score in result.items():
    print('='*56)
    print(m_name)
    print('-'*56)
    print(m_score)

In [ ]:
# 선언
#model_svm = SVC()

# 파라미터 선언
#param = {'C':[0.01,1,10],
         #'gamma': [0.01,0.1,1]}

# Grid Search 모델 선언
#model_svc = GridSearchCV(model_svm,
                         #param,
                         #cv=10,
                         #scoring='accuracy')
            #model_svc.fit(x_train_s, y_train)
            
# 최적파라미터, 성능 확인
#print(model_svc.best_params_)
#print(model_svc.best_score_)

In [ ]:
model = XGBClassifier(eval_metric='mlogloss', random_state=2022)
param = {'max_depth':range(4,7),
         'n_estimators':range(60,81,10)}
model_xgb = GridSearchCV(model, param, cv=10, scoring='accuracy')

In [ ]:
# d-tree
from sklearn.tree import plot_tree
plt.figure(figsize=(25,10))
plot_tree(model_dt.best_estimator_,
         filled=True,
         feature_names=list(x),
         fontsize=8,
         class_names=['No', 'Yes'])
plt.show()

# 딥러닝

In [ ]:
# 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model2 = Sequential()
model2.add(Dense(32, activation='relu', input_shape=(81,)))
model2.add(Dropout(0.3))
model2.add(Dense(16, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
# es
# monitor : EalryStopping의 기준이 되는 값('val_loss'를 입력하면 val_loss가 더 이상 감소되지 않을 경우 적용)
# min_delta : 개선된 것으로 간주하기 위한 최소한의 변화량
# patience : monitor되는 값의 개선이 없을 때 최적의 값을 기준으로 몇 번의 epoch를 더 진행할 지 정하는 값
# verbose : 화면에 상태 표시
# mode : monitor가 최소가 되어야 하는 지, 최대가 되어야 하는지 ('val_accuracy'는 max, 'val_loss'는 min)

# mc
# filepath : 모델을 저장할 경로를 입력합니다. '파일명.h5'
# 모델을 저장할 때, 기준이 되는 값을 지정합니다. ('val_loss'를 입력하면 validation set의 loss가 가장 적을 때 저장)
# verbose : 화면에 상태 표시
# save_best_only : True, False
# mode : auto로 할 경우, 모델이 알아서 min, max 판단하여 모델 저장

In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']) 
es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)
mc = ModelCheckpoint('my_checkpoint.h5', monitor='val_loss', save_best_only=True, verbose=1)

In [ ]:
history = model2.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          epochs=20, 
          callbacks=[es, mc],
          batch_size=16,
          verbose=1)

In [ ]:
performance = pd.DataFrame(model2.history.history)
plt.plot(performance[['loss','val_loss']])
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epochs')
plt.show()

In [ ]:
plt.plot(performance[['loss','val_loss', 'accuracy','val_accuracy']])
plt.legend(['loss','val_loss', 'accuracy','val_accuracy'])
plt.xlabel('Epochs')
plt.show()

In [ ]:
performance_q = pd.DataFrame(model_q.history.history)

plt.plot(performance_q[['accuracy', 'val_accuracy']])
plt.legend(['accuracy', 'val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.show()